# Loading Dateset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# importing required modules
from zipfile import ZipFile
  
# specifying the zip file name
file_name = "/content/drive/MyDrive/Facial Expression/images.zip"
  
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    # zip.printdir()  
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

In [ ]:
import os
len(os.listdir(r'./images/train/happy'))

# Importing Libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os

# important important deep learning Library

from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Activation,MaxPooling2D
from keras.models import Model,Sequential
from tensorflow.keras.optimizers import SGD,RMSprop
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import tensorflow as tf

# Creating Train / validation data sets
Using Image Generators

In [ ]:
batch_size = 128

picture_size = 48
folder_path = r'./images/'

datagen_train  = ImageDataGenerator()
datagen_val = ImageDataGenerator()

train_set = datagen_train.flow_from_directory(folder_path+"train",
                                              target_size = (picture_size,picture_size),
                                              color_mode = "grayscale",
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              shuffle=True)


test_set = datagen_val.flow_from_directory(folder_path+"validation",
                                              target_size = (picture_size,picture_size),
                                              color_mode = "grayscale",
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              shuffle=True)

# Creating the model

In [ ]:
classes = 7
model = Sequential()

#1st CNN layer
model.add(Conv2D(64, (3, 3), padding='valid', input_shape=(48, 48, 1), kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#2nd CNN layer
model.add(Conv2D(128, (5, 5), padding='valid', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#3rd CNN layer
model.add(Conv2D(256, (3, 3), padding='valid'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#4th CNN layer
# model.add(Conv2D(512, (3, 3), padding='valid'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

model.add(Flatten())

#Fully connected 1st layer
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))


# Fully connected layer 2nd layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(classes, activation='softmax'))


# opt = Adam(learning_rate=0.001)
model.compile(optimizer="Adam", loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()


# Adding Checkpoints to save Best Epoch, Early Stopping and Learning Rate Optimization 

In [ ]:
checkpoint = ModelCheckpoint(
    "./model-{val_accuracy:.4f}.h5", monitor='val_accuracy', save_best_only=True, mode='max')

early_stopping = EarlyStopping(monitor='val_loss',
                               min_delta=0,
                               patience=10,
                               verbose=1,
                               restore_best_weights=True
                               )

reduce_learningrate = ReduceLROnPlateau(monitor='val_loss',
                                        factor=0.2,
                                        patience=3,
                                        verbose=1,
                                        min_delta=0.00001)

callbacks_list = [ checkpoint]

epochs = 100

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])


In [ ]:
weights = {
    0: np.random.rand(),
    1: np.random.rand(),
    2: np.random.rand(),
    3: np.random.rand(),
    4: np.random.rand(),
    5: np.random.rand(),
    6: np.random.rand(),
}

history = model.fit(train_set,
                              steps_per_epoch=train_set.n//train_set.batch_size,
                              epochs=epochs,
                              validation_data=test_set,
                              validation_steps=test_set.n//test_set.batch_size,
                              callbacks=callbacks_list
                            #   class_weights = weights
                              )


In [ ]:
model.evaluate_generator(test_set)
test_model = tf.keras.models.load_model('/content/model-0.5922.h5')
print(test_model.evaluate(test_set))
print(test_model.evaluate(train_set))

